In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import os
import glob 
from pandas import ExcelWriter
import re
plt.style.use('ggplot')

In [ ]:
# Column/row numbers in case doodle changes the output format
OK_VALUE = 'OK' # The name of a "yes" vote (might need editing if you have "maybe" votes too)
PARTICIPANT_NAME_COLUMN_IDX = 0 # The column where the participant names are
DATA_START_COLUMN_IDX= PARTICIPANT_NAME_COLUMN_IDX + 1 # The column where the votes start
DATE_START_ROW_IDX = 2 # The row where the dates and times start
DATE_END_ROW_IDX = DATE_START_ROW_IDX + 2


def repeatDate(row):
    previousDate = None
    newRow = []
    for v in row:
        if type(v) == str and v:
            previousDate = v
        newRow.append(previousDate)
    return pd.Series(newRow)



def replaceOK(row):
    return row.apply(lambda v: v if v != OK_VALUE else row.iloc[PARTICIPANT_NAME_COLUMN_IDX])

In [ ]:
FILENAME='Doodle'
doodle = pd.read_excel(f'{FILENAME}.xls')

In [ ]:
# Repeat dates in each cell they apply to
doodle.iloc[DATE_START_ROW_IDX:DATE_END_ROW_IDX] = doodle.iloc[DATE_START_ROW_IDX:DATE_END_ROW_IDX].apply(repeatDate, axis=1)
# Replace OK with participant name and compact table by removing NaN
doodle = doodle.apply(replaceOK, axis=1)
doodle


In [ ]:
doodle = doodle.iloc[DATE_START_ROW_IDX:, DATA_START_COLUMN_IDX:] # keep only relevant rows
doodle = doodle.dropna(how='all').transpose().reset_index(drop=True) # Drop NaN rows
doodle = doodle[doodle.iloc[:, -1] > 0] # Keep only count > 0
doodle = doodle.drop(doodle.columns[-1], axis=1) # Drop "count" column
doodle = doodle.apply(lambda r: r.dropna().reset_index(drop=True), axis=1) # Column compaction, remove nan values

# Name participants
dateColumns = ['Month', 'Day', 'Time']
participants = [f'Participant {i+1}' for i in range(len(doodle.columns)-len(dateColumns))]
# Rename columns
doodle.columns = dateColumns + participants
print('Final Compacted Doodle:')
doodle

In [ ]:
excelWriter = pd.ExcelWriter(f'{FILENAME}_compacted.xls')
doodle.to_excel(excelWriter, index=False)
excelWriter.save()